In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [3]:
import malaya

model = malaya.translation.ms_en.transformer(model = 'noisy-base')

/home/ubuntu/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


Downloading:   0%|          | 0.00/233M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

2022-10-04 14:19:01.611948: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-04 14:19:01.927415: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-04 14:19:01.927497: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-10-04 14:19:01.927531: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-10-04 14:19:01.927793: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-10-04 14:19:01.927920: I

In [4]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
left = [unidecode(s) for s in ms]
right = [unidecode(s) for s in eng]

In [5]:
from tqdm import tqdm

batch_size = 2

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

  0%|                                                                                                   | 0/499 [00:00<?, ?it/s]2022-10-04 14:19:08.985135: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 53002240 exceeds 10% of free system memory.
2022-10-04 14:19:09.092247: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 53002240 exceeds 10% of free system memory.
2022-10-04 14:19:09.092284: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 53002240 exceeds 10% of free system memory.
2022-10-04 14:19:09.179784: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 53002240 exceeds 10% of free system memory.
2022-10-04 14:19:09.179812: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 53002240 exceeds 10% of free system memory.
100%|█████████████████████████████████████████████████████████████████████████████████████████| 499/499 [02:31<00:00,  3.29it/s]


In [6]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [7]:
refs = [filtered_right]
sys = filtered_left

In [8]:
r = bleu.corpus_score(sys, refs)
r.__dict__

{'name': 'BLEU',
 'score': 40.38461101332913,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '71.4/48.2/34.6/25.1 (BP = 0.971 ratio = 0.971 hyp_len = 22889 ref_len = 23570)',
 'bp': 0.9706859594192069,
 'counts': [16347, 10562, 7226, 5003],
 'totals': [22889, 21892, 20895, 19898],
 'sys_len': 22889,
 'ref_len': 23570,
 'precisions': [71.41858534667307,
  48.245934587977345,
  34.582435989471165,
  25.143230475424666],
 'prec_str': '71.4/48.2/34.6/25.1',
 'ratio': 0.9711073398387781}

In [9]:
chrf.corpus_score(sys, refs)

chrF2++ = 63.31